## Library

In [12]:
import datetime
import time
import os
import signal
import configparser  # for .ini file
# for controlling chrome browser
from selenium import webdriver
from selenium.webdriver.common.by import By
# for saving as xlsx file
from openpyxl import Workbook
# for UI
from tkinter import *
from tkinter import messagebox
import tkinter.ttk

"""
selenium: 4.11.2,  
openpyxl: 3.0.9
"""

'\nselenium: 4.11.2,  \nopenpyxl: 3.0.9\n'

.ini file is for convenience through saving report name, storage path.

In [7]:
# Dealing with 'ini' file
config = configparser.ConfigParser()
config.read('.\\conf.ini')

['.\\conf.ini']

In [8]:
def clicked():
    # Forming a Excel Sheet
    write_xl = Workbook()
    write_sheet = write_xl.active
    write_sheet['A1'] = "#"
    write_sheet['B1'] = "Company"
    write_sheet['C1'] = "Title"
    write_sheet['D1'] = "Notice Date"
    write_sheet['E1'] = "Memo"

    # Warning: When saving a excel file, it can rewrite on the original file.
    msg = messagebox.askquestion("경고", "'Data.xlsx'로 저장됩니다. 그대로 진행하시겠습니까? (같은 위치에 동일한 이름의 파일이 있는지 확인해주세요. 덮어쓰여질 수 있습니다.)")
    if msg == 'no':
        root.destroy()
    else:
        pass
    # # If the saving mode is on, save the report_name, storage_path in ini
    if int(config.get("Check_Savings", "report_name")) == 1:
        config.set("DB", "report_name", txt1.get())
        config.set("Check_Savings", "report_name", '%s' % check_var1.get())
        with open('conf.ini', 'w') as handle:
            config.write(handle)
    else:
        pass
    if int(config.get("Check_Savings", "storage_path")) == 1:
        config.set("DB", "storage_path", txt5.get())
        config.set("Check_Savings", "storage_path", '%s' % check_var3.get())
        with open('conf.ini', 'w') as handle:
            config.write(handle)
    # 4 quarters
    base_date = ""
    if which_quarter.get() == "1분기":
        base_date = txt4.get() + "-03-31"
    elif which_quarter.get() == "2분기":
        base_date = txt4.get() + "-06-30"
    elif which_quarter.get() == "3분기":
        base_date = txt4.get() + "-09-30"
    elif which_quarter.get() == "4분기":
        base_date = txt4.get() + "-12-31"
    else:
        root.destroy()

    # Go To Google, first
    driver = webdriver.Chrome()
    driver.get("https://www.google.com/")
    driver.implicitly_wait(10)
    # Go To Dart
    url = "https://dart.fss.or.kr/dsab007/main.do?option=corp"  # Dart searching address
    driver.get(url)
    time.sleep(1)
    # Let's crawl
    report_name = txt1.get()  # search for
    date_from = txt2.get()  # from
    date_to = txt3.get()  # to
    driver.execute_script("toggleDetail('open');")
    time.sleep(1)
    driver.find_element(By.NAME, 'reportName2').send_keys(report_name)  # Input report name in blank
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="startDate"]').clear()   # Setting from when to when
    driver.find_element(By.XPATH, '//*[@id="startDate"]').send_keys(('').join(date_from.split('-')))
    driver.find_element(By.XPATH, '//*[@id="endDate"]').clear()
    driver.find_element(By.XPATH, '//*[@id="endDate"]').send_keys(('').join(date_to.split('-')))
    time.sleep(1)
    button1 = driver.find_element(By.CLASS_NAME, 'btnArea')
    button1_1 = button1.find_element(By.CLASS_NAME, 'btnSearch')
    button1_1.click()
    time.sleep(5)
    # Let's read reports
    monitoring_number = int(driver.find_element(By.XPATH, "//*[@id='maxResultsCb']/option[@selected]").text)  # the number of lists
    end_page = len(driver.find_elements(By.XPATH, "//*[@id ='psWrap']/div[2]/ul/li"))  # the number of pages we are able to look at
    # Setting the initial xpath number
    counting_number = 1
    for t in range(1, end_page + 1):
        page = driver.find_element(By.XPATH, "//*[@id ='psWrap']/div[2]/ul/li[%d]/a" % t)
        if page.text == "1":
            counting_number = t
        else:
            pass
    # Searching the documents(Main)
    time_to_break = False  # variable with 'break'
    while not time_to_break:
        number_of_iteration = len(driver.find_elements(By.XPATH, "//table[@class = 'tbList']/tbody/tr")) + 1  # the number of lists we are able to look at
        for i in range(1, number_of_iteration):
            date = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[5]" % i)  # notice date. this part is not necessary, actually.
            if datetime.datetime.strptime(('-').join(date.text.split('.')), "%Y-%m-%d") < datetime.datetime.strptime(date_from, "%Y-%m-%d"):
                time_to_break = True
                write_xl.save(f'{txt5.get()}\\Data.xlsx')
                break
            else:
                try:
                    a_list = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[@class = 'tL'][2]" % i)  # Opening the announcement
                    a_list.click()
                    time.sleep(1)
                    # change the window
                    driver.switch_to.window(driver.window_handles[-1])
                    # change the frame(In html-iframe, selenium cannot read data inside)
                    inside_frame = driver.find_element(By.XPATH, "//*[@id='ifrm']")
                    driver.switch_to.frame(inside_frame)

                    # Bring the dividend date
                    time_to_exit_for = False
                    data1, data2 = (None, None)
                    for k in range(1, 5):  # Finding the xpath of the dividend date
                        for j in range(1, 20):
                            data1 = driver.find_element(By.XPATH, "//table[%d]/tbody/tr[%d]/td[1]/span" % (k, j)).text
                            if data1 == "기준일":
                                data2 = driver.find_element(By.XPATH, "//table[%d]/tbody/tr[%d]/td[2]/span" % (k, j)).text
                                time_to_exit_for = True
                                break
                            else:
                                pass
                        if time_to_exit_for:
                            break
                    # Exit from the frame and come back
                    driver.switch_to.default_content()
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    if data1 == "기준일" and data2 == base_date:  # if the dividend date is what we want, bring it
                        number = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td" % i)
                        company_name = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[@class = 'tL'][1]/span/a" % i)
                        title_name = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[@class = 'tL'][2]" % i)
                        notice_date = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[5]" % i)
                        write_sheet.append([number.text, company_name.text, title_name.text, notice_date.text])
                    else:
                        pass
                # there happen some errors(maybe subsidiary announcement), bring it with 'error' memo
                except Exception:
                    number = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td" % i)
                    company_name = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[@class = 'tL'][1]/span/a" % i)
                    title_name = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[@class = 'tL'][2]" % i)
                    notice_date = driver.find_element(By.XPATH, "//table[@class = 'tbList']/tbody/tr[%d]/td[5]" % i)
                    write_sheet.append([number.text, company_name.text, title_name.text, notice_date.text, "Error"])
        # go on with next page
        if number_of_iteration - 1 < monitoring_number or counting_number == end_page:
            write_xl.save(f'{txt5.get()}\\Data.xlsx')
            break
        else:
            counting_number = counting_number + 1
            next_page = driver.find_element(By.XPATH, "//*[@id ='psWrap']/div[2]/ul/li[%d]/a" % counting_number)
            next_page.click()
        if next_page.text == "다음 페이지":
            for tt in range(1, len(driver.find_elements(By.XPATH, "//*[@id ='psWrap']/div[2]/ul/li"))+1):
                if driver.find_element(By.XPATH, "//*[@id ='psWrap']/div[2]/ul/li[%d]/a" % tt).text.isnumeric():
                    if int(driver.find_element(By.XPATH, "//*[@id ='psWrap']/div[2]/ul/li[%d]/a" % tt).text) % 10 == 1:
                        counting_number = tt
                    else:
                        pass
                else:
                    pass
        else:
            pass
        time.sleep(3)
    os.kill(driver.service.process.pid, signal.SIGTERM)  # disconnect with Chrome browser
    root.destroy()  # programme over

## UI

There are two kinds of report name.
1. 현금ㆍ현물배당을위한주주명부폐쇄(기준일)결정
2. 중간(분기)배당을위한주주명부폐쇄(기준일)결정

In [9]:
def enter():  # click the 'check' button with the Enter Key
    clicked()


def report_name_saving():  # Memorizing the report name
    config.set("DB", "report_name", txt1.get())
    config.set("Check_Savings", "report_name", '%s' % check_var1.get())
    with open('conf.ini', 'w') as handle:
        config.write(handle)


def export_path_saving():  # Memorizing the storage path
    config.set("DB", "storage_path", txt5.get())
    config.set("Check_Savings", "storage_path", '%s' % check_var3.get())
    with open('conf.ini', 'w') as handle:
        config.write(handle)


def end_date_today():  # Setting the end date to today
    txt3.delete(0, END)
    if check_var2.get() == 1:
        txt3.insert(END, datetime.datetime.today().strftime("%Y-%m-%d"))
    else:
        pass
    config.set("Check_Savings", "end_date", '%s' % check_var2.get())
    with open('conf.ini', 'w') as handle:
        config.write(handle)


root = Tk()
root.title("Lists of Shareholders Crawling By ByeongJae")
root.geometry("390x300+600+260")
root.resizable(False, False)

label1 = Label(root, text="배당을 위한 주주명부폐쇄, 크롤링 프로그램입니다.")
photo = PhotoImage(file=".\\etc\\image.png")
label2 = Label(root, image=photo)
label2.pack()
label1.pack()
Label(root, text="보고서명:").place(x=15, y=90)
Label(root, text="시작일:").place(x=15, y=160)
Label(root, text="종료일:").place(x=15, y=180)
Label(root, text="yyyy-mm-dd").place(x=35, y=140)
Label(root, text="주주명부 폐쇄 분기").place(x=215, y=140)
Label(root, text="년").place(x=260, y=160)
Label(root, text="저장 경로:").place(x=15, y=240)

txt1 = Entry(root, width=32)
txt2 = Entry(root, width=10)
txt3 = Entry(root, width=10)
txt4 = Entry(root, width=5)
txt5 = Entry(root, width=32)
txt1.place(x=78, y=91)
txt2.place(x=62, y=162)
txt3.place(x=62, y=182)
txt4.place(x=220, y=162)
txt5.place(x=80, y=241)

which_quarter = tkinter.ttk.Combobox(root, width=8, height=10, values=["1분기", "2분기", "3분기", "4분기"])
which_quarter.place(x=220, y=182)
which_quarter.set("분기 선택")

# Memorizing Codes
check_var1 = IntVar()
check_var2 = IntVar()
check_var3 = IntVar()
c1 = Checkbutton(root, text="저장", variable=check_var1, command=report_name_saving)
c2 = Checkbutton(root, text="종료일 오늘로 설정", variable=check_var2, command=end_date_today)
c3 = Checkbutton(root, text="저장", variable=check_var3, command=export_path_saving)
c1.place(x=310, y=88)
c2.place(x=16, y=200)
c3.place(x=310, y=239)
if int(config.get("Check_Savings", "report_name")) == 1:
    txt1.insert(END, config.get("DB", "report_name"))
    c1.toggle()
else:
    txt1.insert(END, "")

if int(config.get("Check_Savings", "storage_path")) == 1:
    txt5.insert(END, config.get("DB", "storage_path"))
    c3.toggle()
else:
    txt5.insert(END, "")

if int(config.get("Check_Savings", "end_date")) == 1:
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    txt3.insert(END, today)
    c2.toggle()
else:
    txt2.insert(END, "")

btn1 = Button(root, text="확인", command=clicked)
root.bind('<Return>', enter)
btn1.place(x=180, y=265)

# The End
root.mainloop()